# Transparent Database Encryption in SQL Server 2019 Standard Edition
SQL Server 2019 will support Transparent Database Encryption to allow customers to encrypt their data-at-rest to stay in compliance with various security regulations while using SQL Server 2019 Standard Edition. More details about this feature can be found [here](https://docs.microsoft.com/en-us/sql/relational-databases/security/encryption/transparent-data-encryption?view=sql-server-ver15). Transparent Data Encryption (TDE) encrypts SQL Server, Azure SQL Database, and Azure SQL Data Warehouse data files, known as encrypting data at rest. 

In this notebook, you will be able to do the following:
1. Create a database on SQL Server 2019 Standard Edition
2. Enable TDE (Transparent Database Encryption) on the new database
3. PAUSE and RESUME encryption scan on the database

The following code snippet creates a database and enables TDE for the database.

In [0]:
USE master;  
GO  
CREATE MASTER KEY ENCRYPTION BY PASSWORD = '<UseStrongPasswordHere>';  
go  
CREATE CERTIFICATE MyServerCert WITH SUBJECT = 'My DEK Certificate';  
go  
CREATE DATABASE dbTDE
GO
USE dbTDE;  
GO  
CREATE TABLE tblTest (SNO int, val varchar(255));
INSERT INTO tblTest VALUES (1, 'TDE Test');
GO
SELECT name,is_encrypted from sys.databases WHERE name = 'dbTDE';
GO
CREATE DATABASE ENCRYPTION KEY  
WITH ALGORITHM = AES_128  
ENCRYPTION BY SERVER CERTIFICATE MyServerCert;  
GO  

Verify that TDE is enabled on the database.

# TDE is NOW available on STANDARD edition

In [9]:
SELECT SERVERPROPERTY('Edition') as Edition
    ,name as [Database Name]
    , CASE is_encrypted WHEN 1 THEN 'YES' ELSE 'NO' END as [Is TDE Enabled]
FROM sys.databases;

(5 rows affected)

Total execution time: 00:00:00.025

Edition,Database Name,Is TDE Enabled
Standard Edition (64-bit),master,NO
Standard Edition (64-bit),tempdb,YES
Standard Edition (64-bit),model,NO
Standard Edition (64-bit),msdb,NO
Standard Edition (64-bit),dbTDE,YES


Now let's disable encryption on the database. As you can see from the output, TDE is disabled on the database.

In [10]:
ALTER DATABASE dbTDE set ENCRYPTION OFF;
WAITFOR DELAY '00:00:15'
SELECT SERVERPROPERTY('Edition') as Edition
    ,name as [Database Name]
    , CASE is_encrypted WHEN 1 THEN 'YES' ELSE 'NO' END as [Is TDE Enabled]
FROM sys.databases;

(5 rows affected)

Total execution time: 00:00:15.073

Edition,Database Name,Is TDE Enabled
Standard Edition (64-bit),master,NO
Standard Edition (64-bit),tempdb,YES
Standard Edition (64-bit),model,NO
Standard Edition (64-bit),msdb,NO
Standard Edition (64-bit),dbTDE,NO


Now let's reenable TDE on the database and pause the encryption scan as soon as it starts.

In [11]:
ALTER DATABASE dbTDE set ENCRYPTION ON;
ALTER DATABASE dbTDE SET ENCRYPTION SUSPEND;

Commands completed successfully.

Total execution time: 00:00:00.051

The status of the encryption scan should say <b>SUSPENDED</b>

In [14]:
SELECT db_name(database_id) as [Database Name]
	, encryption_scan_modify_date
	, encryption_scan_state_desc
from sys.dm_database_encryption_keys


(2 rows affected)

Total execution time: 00:00:00.005

Database Name,encryption_scan_modify_date,encryption_scan_state_desc
tempdb,2019-10-11 04:14:58.997,COMPLETE
dbTDE,2019-10-29 22:48:20.150,SUSPENDED


Let's resume the encryption 

In [15]:
ALTER DATABASE dbTDE SET ENCRYPTION RESUME;

Commands completed successfully.

Total execution time: 00:00:00.012

Check if the encryption is complete

In [16]:
SELECT db_name(database_id) as [Database Name]
	, encryption_scan_modify_date
	, encryption_scan_state_desc
FROM sys.dm_database_encryption_keys;

SELECT SERVERPROPERTY('Edition') as Edition
    ,name as [Database Name]
    , CASE is_encrypted WHEN 1 THEN 'YES' ELSE 'NO' END as [Is TDE Enabled]
FROM sys.databases;

(2 rows affected)

(5 rows affected)

Total execution time: 00:00:00.036

Database Name,encryption_scan_modify_date,encryption_scan_state_desc
tempdb,2019-10-11 04:14:58.997,COMPLETE
dbTDE,2019-10-29 22:58:34.353,COMPLETE


Edition,Database Name,Is TDE Enabled
Standard Edition (64-bit),master,NO
Standard Edition (64-bit),tempdb,YES
Standard Edition (64-bit),model,NO
Standard Edition (64-bit),msdb,NO
Standard Edition (64-bit),dbTDE,YES


You should see the following messages in the SQL Server errorlog which shows PAUSE and RESUME for the encryption scan.


<i> 2019-10-29 22:32:11.100 spid34s      Beginning database encryption scan for database 'dbTDE'. <br>
2019-10-29 22:32:11.150 spid34s      <font color = "red">Database encryption scan for database 'dbTDE' was aborted.</font> Reissue ALTER DB to resume the scan. <br>
2019-10-29 22:32:31.730 spid15s      <font color ="red">Beginning database encryption scan for database 'dbTDE'.</font> <br>
2019-10-29 22:32:41.190 spid15s      Database encryption scan for database 'dbTDE' is complete. </i> <br>